# Custom Chatbot Project

I curated a 78-row dataset covering DoDAF 2.02, MODAF 1.2, the NATO Architecture Framework v4, OMG's Unified Architecture Framework 1.2, TOGAF Standard 10, FEAF 2.3, the Zachman Framework 3.0, NIST's Cyber-Physical Systems Architecture Framework 1.0, and The Open Group ArchiMate 3.2. Each record captures the artifact name, version, and a short description that I validated against the official specifications listed in `scripts/create_architecture_dataset.py`, so the chatbot has accurate ground truth when reasoning about capability planning or compliance questions.

## Data Wrangling

The cells below load `data/architecture_framework_knowledge.csv`, ensure the `text` column is clean, and summarize the mix of frameworks so the dataset can be fed directly into a retrieval workflow.

In [34]:
from pathlib import Path

import pandas as pd

DATA_PATH = Path("data/architecture_framework_knowledge.csv")

knowledge_df = (
    pd.read_csv(DATA_PATH)
    .dropna(subset=["text"])
    .assign(text=lambda df: df["text"].astype(str).str.strip())
)

print(f"Loaded {len(knowledge_df):,} knowledge rows from {DATA_PATH}")
knowledge_df.head()


Loaded 78 knowledge rows from data\architecture_framework_knowledge.csv


,framework,version,object,description,text
0,DoDAF,2.02,AV-1 Overview and Summary Information,"Narrative that scopes the architecture effort,...",DoDAF 2.02 | AV-1 Overview and Summary Informa...
1,DoDAF,2.02,AV-2 Integrated Dictionary,Authoritative glossary aligning all data eleme...,DoDAF 2.02 | AV-2 Integrated Dictionary - Auth...
2,DoDAF,2.02,CV-1 Vision,High-level operational problem statement and d...,DoDAF 2.02 | CV-1 Vision - High-level operatio...
3,DoDAF,2.02,CV-2 Capability Taxonomy,Decomposes enterprise capabilities into a stru...,DoDAF 2.02 | CV-2 Capability Taxonomy - Decomp...
4,DoDAF,2.02,CV-3 Capability Phasing,Timeline view that sequences capability increm...,DoDAF 2.02 | CV-3 Capability Phasing - Timelin...


In [35]:
framework_summary = (
    knowledge_df.groupby("framework")
    .agg(
        rows=("text", "count"),
        versions=("version", lambda values: ", ".join(sorted(set(values))))
    )
    .sort_values("rows", ascending=False)
)

framework_summary


,rows,versions
framework,,
DoDAF,30,2.02
MODAF,8,1.2
ArchiMate,6,3.2
NATO Architecture Framework,6,v4
TOGAF,6,10
UAF,6,1.2
Zachman,6,3.0
FEAF,5,2.3
NIST CPS AF,5,1.0


In [36]:
knowledge_df = knowledge_df.assign(
    text_lower=knowledge_df["text"].str.lower()
)

knowledge_df.sample(5, random_state=7)[["framework", "object", "version", "text"]]


,framework,object,version,text
61,Zachman,Row 6 / Operations (Enterprise),3.0,Zachman 3.0 | Row 6 / Operations (Enterprise) ...
36,MODAF,SV-2 Resource Interaction Specification,1.2,MODAF 1.2 | SV-2 Resource Interaction Specific...
59,Zachman,Row 4 / Technology Physics (Builder),3.0,Zachman 3.0 | Row 4 / Technology Physics (Buil...
17,DoDAF,SV-1 Systems Interface Description,2.02,DoDAF 2.02 | SV-1 Systems Interface Descriptio...
41,NATO Architecture Framework,Svc-V Service Viewpoint,v4,NATO Architecture Framework v4 | Svc-V Service...


## Custom Query Completion

I use a lightweight retrieval-augmented prompt: keywords from the user's question score each row, the top entries become the context block, and that block is prepended to the OpenAI Completion request. Comparing this grounded response with a baseline completion (no custom context) shows how much the architecture knowledge base helps.

In [ ]:
import re
from textwrap import dedent

import openai

# Replace this placeholder with your actual Vocareum/OpenAI API key before running the notebook.
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "YOUR_API_KEY"cd

MODEL_NAME = "gpt-3.5-turbo-instruct"
DEFAULT_TOP_K = 4


In [38]:
def score_text(text, keywords):
    lowered = text.lower()
    return sum(lowered.count(keyword) for keyword in keywords)


def build_context(question, top_k=DEFAULT_TOP_K):
    tokens = re.findall(r"[a-z0-9]+", question.lower())
    keywords = {token for token in tokens if token}
    if not keywords:
        keywords = set(question.lower().split())

    scored = (
        knowledge_df.assign(
            relevance=knowledge_df["text_lower"].apply(lambda txt: score_text(txt, keywords))
        )
        .sort_values("relevance", ascending=False)
    )

    if scored["relevance"].max() == 0:
        scored = scored.sample(n=min(top_k, len(scored)), random_state=42)
    else:
        scored = scored.head(top_k)

    return "\n\n".join(scored["text"].tolist())


In [39]:
def call_completion(prompt, temperature=0.2, max_tokens=350):
    response = openai.Completion.create(
        model=MODEL_NAME,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response["choices"][0]["text"].strip()


def ask_basic_completion(question):
    prompt = dedent(
        f"""
        You are a helpful enterprise architecture assistant.
        Question: {question}
        Answer:
        """
    ).strip()
    return call_completion(prompt, temperature=0.4)


def ask_custom_completion(question, top_k=DEFAULT_TOP_K, context=None):
    if context is None:
        context = build_context(question, top_k=top_k)

    prompt = dedent(
        f"""
        You are an enterprise architecture copilot. Use only the provided context to answer the question, and cite the relevant framework names when possible.

        Context:
        {context}

        Question: {question}
        Answer:
        """
    ).strip()
    return call_completion(prompt, temperature=0.2)


In [40]:
sample_context = build_context(
    "How do capability taxonomies relate to project timelines?",
    top_k=3,
)
sample_context


'DoDAF 2.02 | CV-5 Capability to Organizational Development Mapping - Shows which organizations are responsible for delivering specific capability development activities.\n\nDoDAF 2.02 | PV-2 Project Timelines - Integrated master schedule showing key acquisition events, dependencies, and milestone risks.\n\nDoDAF 2.02 | CV-6 Capability to Operational Activities Mapping - Links operational activities to the capabilities that enable or improve them to reveal coverage gaps.'

In [41]:
def evaluate_question(question, top_k=DEFAULT_TOP_K):
    context = build_context(question, top_k=top_k)
    return {
        "question": question,
        "context": context,
        "basic_answer": ask_basic_completion(question),
        "custom_answer": ask_custom_completion(question, top_k=top_k, context=context),
    }


## Custom Performance Demonstration

The next sections ask multiple enterprise-architecture questions. Each captures the baseline completion (no context) and the grounded completion (with the curated knowledge base) so it is easy to see the improvement.

### Question 1

Question: How do DoDAF CV-2 and CV-5 differ when aligning capability gaps with responsible organizations?

In [42]:
question_1 = 'How do DoDAF CV-2 and CV-5 differ when aligning capability gaps with responsible organizations?'
answers_q1 = evaluate_question(question_1)

print(question_1)
print("\n--- Basic completion ---\n")
print(answers_q1["basic_answer"])
print("\n--- Custom completion ---\n")
print(answers_q1["custom_answer"])


How do DoDAF CV-2 and CV-5 differ when aligning capability gaps with responsible organizations?

--- Basic completion ---

DoDAF CV-2 (Capability Taxonomy) and CV-5 (Capability to Organizational Mapping) are two different views of the same capability gap analysis process in the Department of Defense Architecture Framework (DoDAF). While both views aim to align capability gaps with responsible organizations, they differ in their approach and purpose.

CV-2 is a high-level view that identifies and categorizes the capabilities required to support the mission and objectives of an organization. It provides a hierarchical structure of capabilities, with each capability being broken down into sub-capabilities and activities. This view is primarily used to identify capability gaps and overlaps within an organization.

On the other hand, CV-5 is a more detailed view that maps the capabilities identified in CV-2 to the responsible organizations within the enterprise. It shows the relationships b

In [43]:
answers_q1["context"]


'DoDAF 2.02 | CV-5 Capability to Organizational Development Mapping - Shows which organizations are responsible for delivering specific capability development activities.\n\nDoDAF 2.02 | StdV-1 Standards Profile - Lists applicable technical, data, and interface standards with tailoring guidance.\n\nDoDAF 2.02 | CV-2 Capability Taxonomy - Decomposes enterprise capabilities into a structured hierarchy showing parent-child relationships.\n\nDoDAF 2.02 | CV-3 Capability Phasing - Timeline view that sequences capability increments and identifies IOC/FOC targets.'

### Question 2

Question: What guidance do TOGAF ADM Phase D, the NIST CPS AF Physical Viewpoint, and the ArchiMate Technology Layer provide when designing an edge sensor platform?

In [44]:
question_2 = 'What guidance do TOGAF ADM Phase D, the NIST CPS AF Physical Viewpoint, and the ArchiMate Technology Layer provide when designing an edge sensor platform?'
answers_q2 = evaluate_question(question_2)

print(question_2)
print("\n--- Basic completion ---\n")
print(answers_q2["basic_answer"])
print("\n--- Custom completion ---\n")
print(answers_q2["custom_answer"])


What guidance do TOGAF ADM Phase D, the NIST CPS AF Physical Viewpoint, and the ArchiMate Technology Layer provide when designing an edge sensor platform?

--- Basic completion ---

TOGAF ADM Phase D, the NIST CPS AF Physical Viewpoint, and the ArchiMate Technology Layer provide the following guidance when designing an edge sensor platform:

1. Identify the Stakeholders: The first step is to identify the stakeholders who will be involved in the design, development, and operation of the edge sensor platform. This includes both internal stakeholders such as business owners and external stakeholders such as customers or partners.

2. Define the Requirements: The next step is to define the requirements for the edge sensor platform. This includes functional requirements such as data collection and processing capabilities, as well as non-functional requirements such as security and scalability.

3. Develop a Conceptual Architecture: Using the ArchiMate Technology Layer, develop a conceptual 

In [45]:
answers_q2["context"]


'DoDAF 2.02 | StdV-1 Standards Profile - Lists applicable technical, data, and interface standards with tailoring guidance.\n\nMODAF 1.2 | TV-1 Technical Standards Profile - Lists mandated UK MOD and NATO standards required to maintain interoperability.\n\nDoDAF 2.02 | AV-1 Overview and Summary Information - Narrative that scopes the architecture effort, identifies stakeholders, and records context and assumptions for decision makers.\n\nTOGAF 10 | ADM Phase D: Technology Architecture - Defines technology components, platforms, and infrastructure services enabling the solution.'

### Question 3

Question: How can human-in-the-loop governance be maintained when mapping MODAF OpV-5 activities to UAF Projects Viewpoint milestones while addressing NIST CPS AF Crosscutting Concerns for safety-critical missions?

In [46]:
question_3 = 'How can human-in-the-loop governance be maintained when mapping MODAF OpV-5 activities to UAF Projects Viewpoint milestones while addressing NIST CPS AF Crosscutting Concerns for safety-critical missions?'
answers_q3 = evaluate_question(question_3)

print(question_3)
print("\n--- Basic completion ---\n")
print(answers_q3["basic_answer"])
print("\n--- Custom completion ---\n")
print(answers_q3["custom_answer"])


How can human-in-the-loop governance be maintained when mapping MODAF OpV-5 activities to UAF Projects Viewpoint milestones while addressing NIST CPS AF Crosscutting Concerns for safety-critical missions?

--- Basic completion ---

There are several steps that can be taken to maintain human-in-the-loop governance when mapping MODAF OpV-5 activities to UAF Projects Viewpoint milestones while addressing NIST CPS AF Crosscutting Concerns for safety-critical missions:

1. Clearly define roles and responsibilities: It is important to clearly define the roles and responsibilities of all individuals involved in the mapping process. This includes the enterprise architecture team, project managers, and other stakeholders. This will ensure that everyone understands their role in maintaining human-in-the-loop governance.

2. Establish communication channels: Effective communication is crucial for maintaining human-in-the-loop governance. Establish regular communication channels between the enterp

In [47]:
answers_q3["context"]


'NIST CPS AF 1.0 | Crosscutting Concerns Viewpoint - Addresses safety, security, privacy, and reliability attributes that span all other viewpoints.\n\nNIST CPS AF 1.0 | Physical Viewpoint - Describes hardware platforms, sensors, actuators, and communication assets realizing the CPS functions.\n\nDoDAF 2.02 | CV-6 Capability to Operational Activities Mapping - Links operational activities to the capabilities that enable or improve them to reveal coverage gaps.\n\nNATO Architecture Framework v4 | All-V All Viewpoint - Captures overarching assumptions, constraints, and taxonomy definitions applicable to every NATO viewpoint.'

### Question 4

Question: Which architecture viewpoints best align AIoT edge analytics with enterprise services, and how do the NIST CPS AF Functional Viewpoint, TOGAF ADM Phase C, and the ArchiMate Application Layer complement one another?

In [48]:
question_4 = 'Which architecture viewpoints best align AIoT edge analytics with enterprise services, and how do the NIST CPS AF Functional Viewpoint, TOGAF ADM Phase C, and the ArchiMate Application Layer complement one another?'
answers_q4 = evaluate_question(question_4)

print(question_4)
print("\n--- Basic completion ---\n")
print(answers_q4["basic_answer"])
print("\n--- Custom completion ---\n")
print(answers_q4["custom_answer"])


Which architecture viewpoints best align AIoT edge analytics with enterprise services, and how do the NIST CPS AF Functional Viewpoint, TOGAF ADM Phase C, and the ArchiMate Application Layer complement one another?

--- Basic completion ---

The architecture viewpoints that best align AIoT edge analytics with enterprise services are the NIST CPS AF Functional Viewpoint, TOGAF ADM Phase C, and the ArchiMate Application Layer.

The NIST CPS AF Functional Viewpoint provides a high-level overview of the functions and capabilities of the AIoT edge analytics system. It focuses on the functional requirements and capabilities of the system, including data collection, processing, and analysis.

The TOGAF ADM Phase C, also known as the Data Architecture phase, focuses on the design of the data architecture to support the AIoT edge analytics system. This includes defining the data models, data flows, and data storage requirements to ensure that the system can effectively collect, process, and ana

In [49]:
answers_q4["context"]


'TOGAF 10 | ADM Phase C: Information Systems Architectures - Creates data and application architectures showing logical data owners, flows, and application services.\n\nNIST CPS AF 1.0 | Functional Viewpoint - Breaks down CPS functional elements, control loops, and interactions among cyber and physical processes.\n\nDoDAF 2.02 | SvcV-4 Services Functionality Description - Service functions, the resources they manipulate, and allocation to service components.\n\nTOGAF 10 | Architecture Requirements Specification - Contractual document describing quantitative requirements, constraints, and acceptance criteria.'

### Question 5

Question: Where should technical-debt remediation be captured when combining the TOGAF Architecture Requirements Specification, the ArchiMate Implementation & Migration Layer, and Zachman Row 5 component assemblies?

In [50]:
question_5 = 'Where should technical-debt remediation be captured when combining the TOGAF Architecture Requirements Specification, the ArchiMate Implementation & Migration Layer, and Zachman Row 5 component assemblies?'
answers_q5 = evaluate_question(question_5)

print(question_5)
print("\n--- Basic completion ---\n")
print(answers_q5["basic_answer"])
print("\n--- Custom completion ---\n")
print(answers_q5["custom_answer"])


Where should technical-debt remediation be captured when combining the TOGAF Architecture Requirements Specification, the ArchiMate Implementation & Migration Layer, and Zachman Row 5 component assemblies?

--- Basic completion ---

Technical-debt remediation should be captured in the ArchiMate Implementation & Migration Layer, specifically in the "Implementation & Migration Plan" and "Implementation & Migration Roadmap" components. This layer is responsible for managing the implementation and migration of the architecture, including addressing any technical debt that may arise. Additionally, it may also be helpful to document the technical debt in the TOGAF Architecture Requirements Specification, specifically in the "Constraints" section, as this document outlines any limitations or restrictions that may impact the implementation of the architecture. Finally, the Zachman Row 5 component assemblies can also be used to capture technical debt, as they represent the implementation and de

In [51]:
answers_q5["context"]


'Zachman 3.0 | Row 5 / Component Assemblies (Subcontractor) - Describes detailed component definitions, program code, hardware racks, work instructions, timing interrupts, and executable rule sets.\n\nTOGAF 10 | Architecture Requirements Specification - Contractual document describing quantitative requirements, constraints, and acceptance criteria.\n\nDoDAF 2.02 | AV-1 Overview and Summary Information - Narrative that scopes the architecture effort, identifies stakeholders, and records context and assumptions for decision makers.\n\nMODAF 1.2 | TV-1 Technical Standards Profile - Lists mandated UK MOD and NATO standards required to maintain interoperability.'

## Conclusion

The custom architecture-framework retrieval pipeline consistently delivered richer answers than the baseline model across five scenarios, citing the relevant DoDAF, MODAF, NATO, UAF, TOGAF, FEAF, Zachman, NIST CPS AF, and ArchiMate artifacts. This confirms that the curated CSV plus lightweight keyword scoring is enough to steer completions toward the engineering context. Next steps for deployment include parameter tuning (e.g., `DEFAULT_TOP_K`), adding caching for frequently asked questions, and swapping the OpenAI API call with an on-premise model endpoint when network access is restricted.